In [3]:
from xml.dom import minidom
import json 

In [23]:
card4l_xml = 'card4l_meta.xml'
card4l = minidom.parse(card4l_xml)

In [36]:
geographicalBoundingBox = card4l.getElementsByTagName('GeographicalBoundingBox')
west=0
east=0
south=0
north=0
print(geographicalBoundingBox[0].tagName)
print(geographicalBoundingBox[0].nodeName)
print(geographicalBoundingBox[0].nodeType)
print(geographicalBoundingBox[0].nodeValue)
for index in range(len(geographicalBoundingBox)):
    if (geographicalBoundingBox[index].attributes['corner'].value == 'LL'):
        west = geographicalBoundingBox[index].getElementsByTagName('Longitude')[0].childNodes[0].data
        south = geographicalBoundingBox[index].getElementsByTagName('Latitude')[0].childNodes[0].data
        print(geographicalBoundingBox[index].attributes['corner'].value)
        print(west)
        print(south)
    if (geographicalBoundingBox[index].attributes['corner'].value == 'UR'):
        east = geographicalBoundingBox[index].getElementsByTagName('Longitude')[0].childNodes[0].data
        north = geographicalBoundingBox[index].getElementsByTagName('Latitude')[0].childNodes[0].data
        print(geographicalBoundingBox[index].attributes['corner'].value)
        print(east)
        print(north)

GeographicalBoundingBox
GeographicalBoundingBox
1
None
UR
150.0
-25.0
LL
149.0
-26.0


In [27]:
satelliteName = card4l.getElementsByTagName('SatelliteName')[0].firstChild.data
lia = card4l.getElementsByTagName('LocalIncAngle')
liaFileName = lia[0].getElementsByTagName('FileName')[0].childNodes[0].data
print(liaFileName)
area = card4l.getElementsByTagName('LocalContributingArea')
areaFileName = area[0].getElementsByTagName('FileName')[0].childNodes[0].data
print(areaFileName)
mask = card4l.getElementsByTagName('DataMask')
maskFileName = mask[0].getElementsByTagName('FileName')[0].childNodes[0].data
print(maskFileName)
backScatterData = card4l.getElementsByTagName('BackscatterMeasurementData')
polarization = backScatterData[0].getElementsByTagName('Polarization')[0].childNodes[0].data
polarization_metaFileName = backScatterData[0].getElementsByTagName('FileName')[0].childNodes[0].data
print(polarization_metaFileName)

s1_nrb_034C2E_S26E149_2019_09_17_ANGLE.tif
s1_nrb_034C2E_S26E149_2019_09_17_AREA.tif
s1_nrb_034C2E_S26E149_2019_09_17_MASK.tif
s1_nrb_034C2E_S26E149_2019_09_17_HH.tif


In [8]:
# Create your dictionary class  
class stac_dictionary(dict):  
  
    # __init__ function  
    def __init__(self):  
        self = dict()  
          
    # Function to add key:value  
    def add(self, key, value):  
        self[key] = value

In [32]:
stac = stac_dictionary()
stac.add('stac_version', '1.0.0-beta.2')
stac.add('type', 'Feature')
stac.add('stac_extensions', ["sat", "sar", "projection"])
stac.add('bbox', [west, south, east, north])
properties = stac_dictionary()
properties.add('constellation', satelliteName)
stac.add('properties', properties)
assets = stac_dictionary()
angle = stac_dictionary()
angle.add('description', 'local incidence angle')
angle.add('title', liaFileName)
assets.add('angle', angle)
area = stac_dictionary()
area.add('description', 'normalized scattering area')
area.add('title', areaFileName)
assets.add('area', area)
mask = stac_dictionary()
mask.add('description', 'data mask')
mask.add('title', maskFileName)
assets.add('mask', mask)
polarization_meta = stac_dictionary()
polarization_meta.add('description', 'Polarization '+polarization)
polarization_meta.add('title', polarization_metaFileName)
polarization_meta.add('sar:polarization', polarization)
assets.add(polarization, polarization_meta)
metadata = stac_dictionary()
metadata.add('title', card4l_xml)
assets.add('metadata', metadata)
stac.add('assets', assets)
print(stac)

{'stac_version': '1.0.0-beta.2', 'type': 'Feature', 'stac_extensions': ['sat', 'sar', 'projection'], 'bbox': ['149.0', '-26.0', '150.0', '-25.0'], 'properties': {'constellation': 'Sentinel-1'}, 'assets': {'angle': {'description': 'local incidence angle', 'title': 's1_nrb_034C2E_S26E149_2019_09_17_ANGLE.tif'}, 'area': {'description': 'normalized scattering area', 'title': 's1_nrb_034C2E_S26E149_2019_09_17_AREA.tif'}, 'mask': {'description': 'data mask', 'title': 's1_nrb_034C2E_S26E149_2019_09_17_MASK.tif'}, 'HH': {'description': 'Polarization HH', 'title': 's1_nrb_034C2E_S26E149_2019_09_17_HH.tif', 'sar:polarization': 'HH'}, 'metadata': {'title': 'card4l_meta.xml'}}}


In [33]:
print(json.dumps(stac, indent=2))

{
  "stac_version": "1.0.0-beta.2",
  "type": "Feature",
  "stac_extensions": [
    "sat",
    "sar",
    "projection"
  ],
  "bbox": [
    "149.0",
    "-26.0",
    "150.0",
    "-25.0"
  ],
  "properties": {
    "constellation": "Sentinel-1"
  },
  "assets": {
    "angle": {
      "description": "local incidence angle",
      "title": "s1_nrb_034C2E_S26E149_2019_09_17_ANGLE.tif"
    },
    "area": {
      "description": "normalized scattering area",
      "title": "s1_nrb_034C2E_S26E149_2019_09_17_AREA.tif"
    },
    "mask": {
      "description": "data mask",
      "title": "s1_nrb_034C2E_S26E149_2019_09_17_MASK.tif"
    },
    "HH": {
      "description": "Polarization HH",
      "title": "s1_nrb_034C2E_S26E149_2019_09_17_HH.tif",
      "sar:polarization": "HH"
    },
    "metadata": {
      "title": "card4l_meta.xml"
    }
  }
}


In [37]:
productAttributes = card4l.getElementsByTagName('ProductAttributes')
for title in productAttributes:
        nodes = title.childNodes
        for node in nodes:
            if (node.nodeType == node.ELEMENT_NODE) and (node.tagName == 'GeographicalBoundingBox'):
                print(node.attributes['corner'].value)

UL
UR
LL
LR
